# Pre-Process Decagon Data and Save Using Pickle

## 1. Analysis Nodes, Edges, Node Features and Edge Types

In [ ]:
import csv

In [ ]:
# set data paths
pp_f = "data_decagon_sample/bio-decagon-ppi.csv"
dd_f = "data_decagon_sample/bio-decagon-combo.csv"
dp_f = "data_decagon_sample/bio-decagon-targets.csv"
ds_f = "data_decagon_sample/bio-decagon-mono.csv"


In [ ]:
# original data collection holder
p_set = set()                 # protein nodes
d_set = set()                 # drug nodes
combo_set = set()             # edge types
mono_set = set()              # additional drug features

# read file to list
pp_list, ddt_list, dp_list, ds_list = [], [], [], []   # lists of tuples

# set temporary variable
a, b, c = 0, 0, 0

### 1.1 protein-protein association network

Includes:

- physical interaction
- functional associations

Notation:

- node: protein
- edge: association or interaction

Number:

- 19081 nodes (min: 1; max: 110255169)
- 715612 edges

In [ ]:
with open(pp_f, 'r') as f:
    ppi = csv.reader(f)
    next(ppi)
    for [g1, g2] in ppi:
        a, b = int(g1), int(g2)
        p_set.add(a)
        p_set.add(b)
        pp_list.append((a, b))
print("{:d} protein nodes indexed from {:d} to {:d}.".format(len(p_set), min(p_set), max(p_set)))
print("{:d} protein-protein edges".format(len(pp_list)))

### 1.2 drug-drug association network with side effects

!! TIME COSTY SECTION

Notation:

- node: drug
- edge with typies: type of side effects that are associated with drug pairs

Number:

- 645 nodes
- 63473 edges
- 1317 edge types

In [ ]:
with open("data_decagon_sample/bio-decagon-combo.csv", "r") as f:
    ppi = csv.reader(f)
    next(ppi)
    for [d1, d2, t, n] in ppi:
        a, b, c = int(t.split('C')[-1]), int(d1.split('D')[-1]), int(d2.split('D')[-1])
        combo_set.add(a)
        d_set.add(b)
        d_set.add(c)
        ddt_list.append((b, c, a))
print("{:d} drug nodes indexed from {:d} to {:d}".format(len(d_set), min(d_set), max(d_set)))
print("{:d} drug-drug edges with {:d} edge types ".format(len(ddt_list), len(combo_set)))

### 1.3 drug-protein association network

Notation:

- node: drug and genes(encode protein)
- edge: gene is targeted by drug

Number:

- 3932 nodes: 284 drug; 3648 gene
- 18690 edges

In [ ]:
with open("data_decagon_sample/bio-decagon-targets.csv", "r") as f:
    ppi = csv.reader(f)
    next(ppi)
    for [d, p] in ppi:
        a, b = int(d.split('D')[-1]), int(p)
        d_set.add(a)
        p_set.add(b)
        dp_list.append((a, b))
print("{:d} drug-protein edges".format(len(dp_list)))
print("{:d} proteins not in the ppi network".format(p_set.__len__() - 19081))
print("{:d} drugs not in the drug-drug network".format(d_set.__len__() - 645))


### 1.4 single drug side effects

Notation:

- node: drug and side-effect
- edge: side effect caused by drug

Number:

- 10825 nodes: 640 drug; 10184 side effect
- 174978 edges

In [ ]:
temp = d_set.__len__()
with open("data_decagon/bio-decagon-mono.csv", "r") as f:
    ppi = csv.reader(f)
    next(ppi)
    for [d, e, n] in ppi:
        a, b = int(e.split('C')[-1]), int(d.split('D')[-1])
        mono_set.add(a)
        d_set.add(b)
        ds_list.append((b, a))
print("{:d} drugs not in the drug-drug network."
      .format(d_set.__len__() - temp))
print("{:d} single drug side effects in drug pair side effects set."
      .format((combo_set & mono_set).__len__()))


In [ ]:
# the numbers
num_gene = p_set.__len__()
num_drug = d_set.__len__()
num_edge_type = combo_set.__len__()
num_drug_additional_feature = mono_set.__len__()

print("Summary: ")
print(" -> Protein Node              : {:d}".format(num_gene))
print(" -> Drug    Node              : {:d}".format(num_drug))
print(" -> Drug    Pair  Side  Effect: {:d}".format(num_edge_type))
print(" -> Single  Drug  Side  Effect: {:d}".format(num_drug_additional_feature))

## 2. Index Drug Nodes, Protein Nodes, Drug Additional Features and Drug-Drug Edge Types

Note: Using gene presents its encode protein

In [ ]:
import scipy.sparse as sp

### Re-indexing Stragy

xxx\_to\_new is a one-row sparse matrix:

- index: real drug/protein/side effect index  
- value: its new index 

xxx\_to\_old is a list:

- index: the re-allocated index
- value: original index


In [ ]:
# protein
gene_to_old = list(p_set)
gene_to_new = sp.csr_matrix((range(num_gene), ([0] * num_gene, gene_to_old)))

# drug
drug_to_old = list(d_set)
drug_to_new = sp.csr_matrix((range(num_drug), ([0] * num_drug, drug_to_old)))

# drug pair side effect
edge_type_to_old = list(combo_set)
edge_type_to_new = sp.csr_matrix((range(num_edge_type), ([0] * num_edge_type, edge_type_to_old)))

# single drug side effect
side_effect_to_old = list(mono_set)
side_effect_to_new = sp.csr_matrix((range(num_drug_additional_feature), ([0] * num_drug_additional_feature, side_effect_to_old)))


## 3. Construct Sparse Adjacency Matrix

### 3.1 gene-gene adjacency matrix



In [ ]:
import numpy as np

In [ ]:
# set temporary variable
r, c, array_length = [], [], len(pp_list)

In [ ]:
for i in range(array_length):
    r.append(gene_to_new[0, pp_list[i][0]])
    c.append(gene_to_new[0, pp_list[i][1]])
    
gene_adj = sp.csr_matrix(([1] * array_length, (r, c)), shape=(num_gene, num_gene))

gene_degrees = np.array(gene_adj.sum(axis=0)).squeeze()


### 3.2 drug-gene & gene-drug adjacency matrix


In [ ]:
# set temporary variable
r, c, array_length = [], [], len(dp_list)

In [ ]:
for i in range(array_length):
    r.append(drug_to_new[0, dp_list[i][0]])
    c.append(gene_to_new[0, dp_list[i][1]])

drug_gene_adj = sp.csr_matrix(([1] * array_length, (r, c)), shape=(num_drug, num_gene))
gene_drug_adj = drug_gene_adj.transpose(copy=True)
    

### 3.3 drug-drug adjacency matrix list 

Note: this list is indexed by drug-drug edge type index

!! TIME COSTY SECTION

In [ ]:
# set temporary variable
# r: key is the index of edge type, value is a list of two lists [drug list, drug list]
r = {}        
array_length = len(ddt_list)


In [ ]:
# build drug-drug network by edge types
for i in range(array_length):
    c = edge_type_to_new[0, ddt_list[i][2]]
    if c not in r:
        r[c] = [drug_to_new[0, ddt_list[i][0]]], [drug_to_new[0, ddt_list[i][1]]]
    else:
        r[c][0].append(drug_to_new[0, ddt_list[i][0]])
        r[c][1].append(drug_to_new[0, ddt_list[i][1]])

# build adjacency matrix
drug_drug_adj_list = []
for i in range(num_edge_type):
    drug_drug_adj_list.append(sp.csr_matrix(([1] * len(r[i][0]), (r[i][0], r[i][1])), shape=(num_drug, num_drug)))

drug_degrees_list = [np.array(drug_adj.sum(axis=0)).squeeze() for drug_adj in drug_drug_adj_list]

                

## 4. Generate Feature Vector

### 4.1 gene one-hot vector

In [ ]:
import decagon.utility.preprocessing as preprocessing

In [ ]:
gene_feat = sp.identity(num_gene)

gene_nonzero_feat, gene_num_feat = gene_feat.shape
gene_feat = preprocessing.sparse_to_tuple(gene_feat.tocoo())

### 4.2 drug feature vector with additional feature

In [ ]:
# index feature
r, c = list(range(num_drug)), list(range(num_drug))

# additional feature append to index feature
for (a, b) in ds_list:
    r.append(drug_to_new[0, a])
    c.append(side_effect_to_new[0, b] + num_drug)

array_length = num_drug + len(ds_list)
drug_feat_sparse = sp.csr_matrix(([1] * array_length, (r, c)), shape=(num_drug, num_drug + num_drug_additional_feature))

drug_nonzero_feat, drug_num_feat = drug_feat_sparse.shape[1], np.count_nonzero(drug_feat_sparse.sum(axis=0))
drug_feat = preprocessing.sparse_to_tuple(drug_feat_sparse.tocoo())


### 4.3 Visualization of side effects

In [ ]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
%matplotlib inline


In [ ]:
# dimension reduction
drug_embedded = TSNE(n_components=2).fit_transform(drug_feat_sparse.toarray())


In [ ]:
# random selecting five side effects for visualization
np.random.seed(19680801)
num_show_edge_index = 4
num_show_edge = 60

selected_edge_types = np.random.choice(num_edge_type, num_show_edge_index, replace=False)
selected_edge_types.tolist()

In [ ]:
plt.figure(figsize=(12, 12), dpi= 80, facecolor='w', edgecolor='k')
title = "Drug Embedding Before Training"
col = ['r', 'g', 'b', 'k', 'c', 'm', 'y']

plt.scatter(drug_embedded[:, 0], drug_embedded[:, 1], s=6, label='drug')
for i in range(num_show_edge_index):
    edge = selected_edge_types[i]
    r, c = drug_drug_adj_list[edge].nonzero()
    
    if len(r) > num_show_edge:
        selected_drug_pair_index = np.random.choice(len(r), num_show_edge, replace=False).tolist()
    else:
        selected_drug_pair_index = list(range(len(r)))
        
    for j in selected_drug_pair_index:
        [x1, y1] = drug_embedded[r[j], :]
        [x2, y2] = drug_embedded[c[j], :]
        
        plt.plot([x1, x2], [y1, y2], color=col[i], alpha=0.3, marker='*')
    

# setting
plt.title(title)
plt.grid()
plt.savefig("Drug Embedding before training sample.pdf")
plt.show()

## 5. Save to Files

### 5.1 save graph infomation

In [ ]:
import pickle

In [ ]:
with open('data_decagon_sample/graph_num_info.pkl', 'wb') as f:
    pickle.dump([num_gene, num_drug, num_edge_type, num_drug_additional_feature], f)



### 5.2 save sparse matrix

In [ ]:
# gene-gene adj
sp.save_npz("data_decagon_sample/gene-sparse-adj.npz", gene_adj)

# gene-drug adj
sp.save_npz("data_decagon_sample/drug-gene-sparse-adj.npz", drug_gene_adj)
sp.save_npz("data_decagon_sample/gene-drug-sparse-adj.npz", gene_drug_adj)

# drug-drug adj
for i in range(num_edge_type):
    sp.save_npz("".join(["data_decagon_sample/drug-sparse-adj/type_", str(i), ".npz"]), drug_drug_adj_list[i])
    

### 5.3 save drug sparse feature vectors

In [ ]:
# drug feature vector
sp.save_npz("data_decagon_sample/drug-feature-sparse.npz", drug_feat_sparse)
